# **Ingestion del connectivity.json**

##### **01.- Leer archivo JSON usando dataframereader de spark**

In [1]:
#p_file_date = "2025-07-07"
#p_environment = "Production"

StatementMeta(, 0b64e8e3-b8c2-4632-bddf-f2b095ce7af6, 3, Finished, Available, Finished)

In [2]:
%run configuration

StatementMeta(, 0b64e8e3-b8c2-4632-bddf-f2b095ce7af6, 4, Finished, Available, Finished)

In [3]:
%run common_functions

StatementMeta(, 0b64e8e3-b8c2-4632-bddf-f2b095ce7af6, 5, Finished, Available, Finished)

In [4]:
#Importaciones de los tipos de datos para definir el esquema
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, StringType

StatementMeta(, 0b64e8e3-b8c2-4632-bddf-f2b095ce7af6, 6, Finished, Available, Finished)

In [5]:
# Esta es la estructura del df, esto se define para que inferSchema no haga un proceso extra 
# sirve para grandes volumenes de datos, asi se demoran menos en cargar
model_schema = StructType ( fields = [
    StructField("ConnectivityID", IntegerType(), False),
    StructField("USB", StringType(), True),
    StructField("USBConnector", StringType(), True),
    StructField("Bluetooth", StringType(), True),
    StructField("NFC", StringType(), True),
    StructField("SIMCardSlot", StringType(), True)
])

StatementMeta(, 0b64e8e3-b8c2-4632-bddf-f2b095ce7af6, 7, Finished, Available, Finished)

In [6]:
# Ruta del archivo CSV en Azure Blob Storage o ADLS
abfs_path = f"{bronze_folder_path}/{p_file_date}/connectivity.json"

# Leer el archivo CSV desde ABFS
# El delimitador en este caso son de ; hay que agregar la option delimiter
df = spark.read.schema(model_schema) \
                .option("multiline", True) \
                .json(abfs_path)

StatementMeta(, 0b64e8e3-b8c2-4632-bddf-f2b095ce7af6, 8, Finished, Available, Finished)

##### **2.- Cambiar nombres de columnas**

In [7]:
new_cols = ["connectivity_id", "usb", "usb_connector", "bluetooth", "nfc", "sim_card_slot"]

df2 = df.toDF(*new_cols)

StatementMeta(, 0b64e8e3-b8c2-4632-bddf-f2b095ce7af6, 9, Finished, Available, Finished)

##### **3.- Incluir ingestion_date y environment**

In [8]:
from pyspark.sql.functions import lit

df3 = add_ingestion_date(df2) \
        .withColumn("environment", lit(p_environment)) \
        .withColumn("file_date", lit(p_file_date))

StatementMeta(, 0b64e8e3-b8c2-4632-bddf-f2b095ce7af6, 10, Finished, Available, Finished)

In [9]:
%%sql
DROP TABLE if EXISTS connectivity

StatementMeta(, 0b64e8e3-b8c2-4632-bddf-f2b095ce7af6, 11, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

##### **4.- Pasar a delta y enviar a lh_silver**

In [10]:
df3.write.format("delta").mode("overwrite").saveAsTable("connectivity")

StatementMeta(, 0b64e8e3-b8c2-4632-bddf-f2b095ce7af6, 12, Finished, Available, Finished)

In [11]:
%%sql
SELECT * from connectivity
limit(10)

StatementMeta(, 0b64e8e3-b8c2-4632-bddf-f2b095ce7af6, 13, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 9 fields>